<a href="https://colab.research.google.com/github/Razzf/MachineLearningTests/blob/master/alexnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
import cv2
import random
from IPython.display import clear_output
from google.colab.patches import cv2_imshow

from sklearn import model_selection

import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation, BatchNormalization, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential



In [ ]:
DATADIR = "/content/drive/My Drive/color"
CATEGORIES = ["Apple___Apple_scab", "Apple___Black_rot", "Apple___Cedar_apple_rust",
              "Apple___healthy", "Blueberry___healthy", "Cherry_(including_sour)___Powdery_mildew",
              "Cherry_(including_sour)___healthy", "Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot",
              "Corn_(maize)___Common_rust_", "Corn_(maize)___Northern_Leaf_Blight", "Corn_(maize)___healthy",
              "Grape___Black_rot", "Grape___Esca_(Black_Measles)", "Grape___Leaf_blight_(Isariopsis_Leaf_Spot)",
              "Grape___healthy", "Orange___Haunglongbing_(Citrus_greening)", "Peach___Bacterial_spot",
              "Peach___healthy", "Pepper,_bell___Bacterial_spot", "Pepper,_bell___healthy", "Potato___Early_blight",
              "Potato___Late_blight", "Potato___healthy", "Raspberry___healthy", "Soybean___healthy",
              "Squash___Powdery_mildew", "Strawberry___Leaf_scorch", "Strawberry___healthy", 
              "Tomato___Bacterial_spot", "Tomato___Early_blight", "Tomato___Late_blight", "Tomato___Leaf_Mold",
              "Tomato___Septoria_leaf_spot", "Tomato___Spider_mites Two-spotted_spider_mite",
              "Tomato___Target_Spot", "Tomato___Tomato_Yellow_Leaf_Curl_Virus", "Tomato___Tomato_mosaic_virus",
              "Tomato___healthy"]

In [42]:
training_data = []
counter = 0
#CATEGORIES = np.array(CATEGORIES)
CATEGORIES = CATEGORIES[28:]
print(CATEGORIES)

['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']


In [ ]:
IMG_SIZE = 224
def create_training_data():
  for category in CATEGORIES:
    counter = 0
    path = os.path.join(DATADIR, category)
    class_index = CATEGORIES.index(category)
    class_array = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    for img in os.listdir(path):
      counter = counter + 1
      print(class_index, ":", category, ": image number :", counter)
      class_array[class_index] = 1
      img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_ANYCOLOR)
      img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
      new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
      #optional resize (in this case isn't necesary)
      training_data.append((new_array, class_array))
      if counter % 3 == 0:
        clear_output()
      if counter > 1000:
        break
    





In [ ]:
create_training_data()

In [ ]:


random.shuffle(training_data)

X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)



X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y = np.array(y)

In [53]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation, BatchNormalization, Dropout, Flatten, Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.models import Sequential

#Instantiate an empty model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(IMG_SIZE, IMG_SIZE, 3), kernel_size=(11, 11), strides=(4, 4), padding='valid'))
model.add(BatchNormalization())
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding = 'valid'))



# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding = 'valid'))
model.add(BatchNormalization())
model.add(Activation('relu'))
# Max Pooling
model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding = 'valid'))

# Passing it to a Fully Connected layer
model.add(Flatten())
# 1st Fully Connected Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))

# Output Layer
model.add(Dense(1000))
model.add(Activation('softmax'))

model.summary()

# Compile the model
model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 54, 54, 96)        34944     
_________________________________________________________________
batch_normalization_6 (Batch (None, 54, 54, 96)        384       
_________________________________________________________________
activation_40 (Activation)   (None, 54, 54, 96)        0         
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 25, 25, 256)       221440    
_________________________________________________________________
batch_normalization_7 (Batch (None, 25, 25, 256)       1024      
_________________________________________________________________
activation_41 (Activation)   (None, 25, 25, 256)      

In [ ]:
print(X[0])
print(y[0])

In [ ]:
#x_train, x_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=.2)

model.fit(X, y, batch_size=16, epochs=25, validation_split=.2)